In [72]:
#Prueba de actualizacion - Base de dato DNRPA
# Carga de librerias

import datetime
import mysql.connector
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import openpyxl
import pandas as pd
from datetime import datetime
import os

#Variables globales
host = '172.17.22.10'
user = 'Ivan'
password = 'Estadistica123'
database = 'prueba1'

#DF obtenido del sitio DRPNA
df_web = pd.DataFrame() 

In [73]:
# Conexion con BDD 
conn = mysql.connector.connect( host=host, user=user, password=password, database=database)

#Instancia manejadora de BDD

cursor = conn.cursor()

In [74]:
#Busqueda por nombre de TABLA 

table_name = "dnrpa_inscripcion_nacion_auto"

#Creacion de orden en lenguaje MYSQL
select_dates_query = "SELECT * FROM dnrpa_inscripcion_nacion_auto"

#Obtencion del DATAFRAME - uso de Orden creada(MySQL) y Conexion(MySQL).
df = pd.read_sql(select_dates_query,con=conn)


#Busqueda del año mas grande (actual o anterior) --> Aclaracion por posible conflicto en los avisos por cada año nuevo
año_de_comparacion = int(df['Fecha'].max().year)

#Conversion a tipo de fecha - necesario para el tratamiento
df['Fecha'] = pd.to_datetime(df['Fecha'])

#Toma de año actual - Se usara para comparar los dataframes
df_año_actual = df[df['Fecha'].dt.year == año_de_comparacion]
print(df_año_actual)





         Fecha  Buenos_Aires  C_Autonoma_De_BSAS  Catamarca  Cordoba  \
108 2023-01-01       14373.0              9298.0      391.0   5325.0   
109 2023-02-01        8830.0              5616.0      235.0   3368.0   
110 2023-03-01       11349.0              7709.0      339.0   4367.0   
111 2023-04-01        9872.0              6719.0      310.0   3796.0   
112 2023-05-01       11236.0              8280.0      334.0   4126.0   
113 2023-06-01       11316.0              8236.0      369.0   4160.0   
114 2023-07-01           0.0                 0.0        0.0      0.0   
115 2023-08-01           0.0                 0.0        0.0      0.0   
116 2023-09-01           0.0                 0.0        0.0      0.0   
117 2023-10-01           0.0                 0.0        0.0      0.0   
118 2023-11-01           0.0                 0.0        0.0      0.0   
119 2023-12-01           0.0                 0.0        0.0      0.0   

     Corrientes   Chaco  Chubut  Entre_Rios  Formosa  ...  Rio_

C:\Users\Usuario\AppData\Local\Temp\ipykernel_4840\482719763.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(select_dates_query,con=conn)


In [75]:
from pathlib import Path
import os

# Obtener el directorio actual del notebook
directorio_actual = Path(os.getcwd())

# Obtener la ruta completa del archivo .ipynb
ruta_archivo_ipynb = directorio_actual.joinpath(Path())

# Obtener la dirección del archivo .ipynb
direccion_archivo_ipynb = str(ruta_archivo_ipynb.resolve())

# Mostrar la dirección del archivo
print(direccion_archivo_ipynb)

C:\Users\Usuario\Desktop\scrapingTrabajo\script_DNRPA


In [76]:
#===== Obtencion de datos de la web DROPNA =====#

try:
# Obtener la ruta del directorio actual (donde se encuentra el script)
    directorio_actual = direccion_archivo_ipynb
    ruta_carpeta_files = os.path.join(directorio_actual, 'files')
    ruta_archivo_excel = os.path.join(ruta_carpeta_files, 'registroInscripcionAutoNacion.xlsx')

            #↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓ SELENIUM ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
    driver = webdriver.Chrome()
    driver.get('https://www.dnrpa.gov.ar/portal_dnrpa/estadisticas/rrss_tramites/tram_prov.php?origen=portal_dnrpa&tipo_consulta=inscripciones')
            # Obtener la ventana actual
    ventana_actual = driver.current_window_handle
            
    elemento = driver.find_element(By.XPATH, '//*[@id="seleccion"]/center/table/tbody/tr[2]/td/select')
    # Obtener todas las opciones del elemento select
    opciones = elemento.find_elements(By.TAG_NAME, 'option')

    # Buscar la opción deseada por su valor y hacer clic en ella
    valor_deseado = str(año_de_comparacion) # Valor de la opción que deseas seleccionar

    for opcion in opciones:
        if opcion.get_attribute('value') == valor_deseado:
            opcion.click()
            break
    
    boton = driver.find_element(By.XPATH, '//*[@id="seleccion"]/center/table/tbody/tr[4]/td/input[1]')
    boton.click()
    
    time.sleep(5)
    
    boton_aceptar = driver.find_element(By.XPATH, '//*[@id="seleccion"]/center/center/input')
    boton_aceptar.click()
    
    # Esperar un momento para que se abra la nueva pestaña
    driver.implicitly_wait(5)
    # Cambiar al contexto de la nueva pestaña
    for ventana in driver.window_handles:
        if ventana != ventana_actual:
            driver.switch_to.window(ventana)
    
    time.sleep(5)
    #↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓ ENCONTRAR Y TOMAR LOS DATOS ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
    # Encontrar el elemento <div> con la clase 'grid'
    elemento_div = driver.find_element(By.CLASS_NAME, 'grid')

    # Encontrar la tabla dentro del elemento <div>
    elemento_tabla = elemento_div.find_element(By.TAG_NAME, 'table')

    # Obtener todas las filas de la tabla
    filas = elemento_tabla.find_elements(By.TAG_NAME, 'tr')

    # Lista para almacenar los datos de la tabla
    tabla_datos = []
    
    for fila in filas:
        # Obtener las celdas de cada fila, excluyendo la última columna y la última celda de encabezado
        celdas = fila.find_elements(By.TAG_NAME, 'th') + fila.find_elements(By.TAG_NAME, 'td')[:-1]

        # Lista para almacenar los valores de cada fila
        fila_datos = []

        for celda in celdas:
            valor = celda.text
            if isinstance(valor, str):
                # Verificar si el valor comienza con un número
                if valor.strip() and valor[0].isdigit():
                    try:
                        # Reemplazar el punto decimal por una coma (si es necesario)
                        valor = valor.replace('.', '')
                        # Intentar convertir el valor a float
                        valor = float(valor)
                        print("valor1: ", valor)
                    except ValueError:
                        pass  # Mantener el valor original si no se puede convertir a float
            fila_datos.append(valor)
        print("aca: ", fila_datos)
        # Verificar si la última celda es "Total" y eliminarla
        if fila_datos and fila_datos[-1] == "Total":
            fila_datos.pop()


        tabla_datos.append(fila_datos) 
    
    datos_sin_segunda_fila = tabla_datos[0:1] + tabla_datos[2:]
    # Transponer los datos utilizando pandas
    df = pd.DataFrame(datos_sin_segunda_fila)
    df_transpuesta = df.transpose()
    
    #Conversion de MESES a formato Y-M-D , tipo de dato: datetime
    print(df_transpuesta[0][1:])
    meses = df_transpuesta[0][1:]

    #Donde almacenamos las nuevas fechas
    nuevas_fechas = list()

    for i in range(1, len(meses)+ 1):

        if i < 10:
            fecha_str =  '01-0'+str(i)+"-"+ str(valor_deseado)
        else:
            fecha_str = '01-'+str(i)+"-"+ str(valor_deseado)

        fecha_str = datetime.strptime(fecha_str,'%d-%m-%Y').date()
        nuevas_fechas.append(fecha_str)

    #Reasignacion de fechas
    df_transpuesta[0][1:] = nuevas_fechas
    
    df_web = df_transpuesta #-->  OBTENCION DEL DF CONSTRUIDO PARA SU ANALISIS

except Exception as e:
            # Manejar cualquier excepción ocurrida durante la carga de datos
            print(f"Registro automotor: Ocurrió un error durante la carga de datos: {str(e)}")
            conn.close()  # Cerrar la conexión en caso de error

aca:  ['Provincia / Mes', 'Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic', 'Total']
aca:  []
valor1:  14373.0
valor1:  8830.0
valor1:  11349.0
valor1:  9872.0
valor1:  11236.0
valor1:  11316.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
aca:  ['BUENOS AIRES', 14373.0, 8830.0, 11349.0, 9872.0, 11236.0, 11316.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
valor1:  9298.0
valor1:  5616.0
valor1:  7709.0
valor1:  6719.0
valor1:  8280.0
valor1:  8236.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
aca:  ['C.AUTONOMA DE BS.AS', 9298.0, 5616.0, 7709.0, 6719.0, 8280.0, 8236.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
valor1:  391.0
valor1:  235.0
valor1:  339.0
valor1:  310.0
valor1:  334.0
valor1:  369.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
aca:  ['CATAMARCA', 391.0, 235.0, 339.0, 310.0, 334.0, 369.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
valor1:  5325.0
valor1:  3368.0
valor1:  4367.0

In [77]:
#Manejo de DATAFRAMES AUXILIARES
df_auxiliar_web = df_web.copy()
df_auxiliar_año_actual = df_año_actual.copy()

"""
print(df_auxiliar_web)

print(df_auxiliar_año_actual)

"""

'\nprint(df_auxiliar_web)\n\nprint(df_auxiliar_año_actual)\n\n'

In [78]:
"""
==============================================================================================================

            ***** MANIPULACION DEL DATAFRAME OBTENIDO DE LA WEB *****

==============================================================================================================
    
 """


#Obtencion de las columnas
lista_columnas = list(df_año_actual.columns)
lista_columnas.append("col_auxiliar")

print(lista_columnas)

#Asignar columnas
df_auxiliar_web.columns = lista_columnas


#Eliminar la ultima columna
df_auxiliar_web = df_auxiliar_web.iloc[:, :-1]

#Eliminacion de la primera fila
df_auxiliar_web = df_auxiliar_web.drop(0)


['Fecha', 'Buenos_Aires', 'C_Autonoma_De_BSAS', 'Catamarca', 'Cordoba', 'Corrientes', 'Chaco', 'Chubut', 'Entre_Rios', 'Formosa', 'Jujuy', 'La_Pampa', 'La_Rioja', 'Mendoza', 'Misiones', 'Neuquen', 'Rio_Negro', 'Salta', 'San_Juan', 'San_Luis', 'Santa_Cruz', 'Santa_Fe', 'Sgo_Del_Estero', 'Tucuman', 'Tierra_Del_Fuego', 'Total_Nacion', 'col_auxiliar']


In [84]:
"""
==============================================================================================================

            ***** COMPARACION DE LA BDD CON LOS DATOS OBTENIDOS DE LA WEB *****

==============================================================================================================
    
 """
 
df_auxiliar_web.index = df_auxiliar_año_actual.index

print(df_auxiliar_web.index , df_auxiliar_año_actual.index)

 
valor = df_año_actual.equals(df_auxiliar_web)

print("LOS DATAFRAMES SON IGUALES? ",valor)

igualdad = (df_auxiliar_web.compare(df_año_actual))
 
 
print(igualdad)

Index([108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119], dtype='int64') Index([108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119], dtype='int64')
LOS DATAFRAMES SON IGUALES?  False
Empty DataFrame
Columns: []
Index: []
